VisionReasoner
====

 **VisionReasoner: Unified Visual Perception and Reasoning via Reinforcement Learning**

* Paper: https://arxiv.org/abs/2505.12081

![VisionReasoner Overview](../assets/visionreasoner_overview.jpg)

* Installation

```bash
git clone https://github.com/dvlab-research/VisionReasoner.git VisionReasoner_repo
cd VisionReasoner_repo
conda create -n visionreasoner_test python=3.12
conda activate visionreasoner_test
pip3 install torch torchvision
pip install -r requirements.txt
```

* FlashAttention
If you dont have flash-attention installed, and don't want to install it, change the following lines to `"eager"`

```
# Line 63 - File: VisionReasoner_repo/vision_reasoner/models/vision_reasoner_model.py
attn_implementation="flash_attention_2", # -> "eager"

# Line 11 - File VisionReasoner_repo/vision_reasoner/models/task_router.py
attn_implementation="flash_attention_2", # -> "eager"
```

 * Download the model

```bash
mkdir pretrained_models
cd pretrained_models
git lfs install
git clone https://huggingface.co/Ricky06662/VisionReasoner-7B
git clone https://huggingface.co/Ricky06662/TaskRouter-1.5B
```

In [2]:
import os
import sys

from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

sys.path.append("VisionReasoner_repo/vision_reasoner")
from models.vision_reasoner_model import VisionReasonerModel
from utils import (
    visualize_results_enhanced,
    visualize_pose_estimation_results_enhanced,
    visualize_depth_estimation_results_enhanced,
    visualize_results_video,
    visualize_pose_estimation_results_video,
    visualize_depth_estimation_results_video
)

/home/pyml/anaconda3/envs/visionreasoner_test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_path = 'pretrained_models/VisionReasoner-7B'
task_router_model_path = "pretrained_models/TaskRouter-1.5B"
segmentation_model_path = "facebook/sam2-hiera-large"

task_type = "auto"
# Possible choices for task_type:
#  - "auto"
#  - "detection"
#  - "segmentation"
#  - "counting"
#  - "vqa"
#  - "generation"
#  - "depth_estimation"
#  - "pose_estimation"

hybrid_mode = False
yolo_model_path = "yolov8x-worldv2.pt"
refer_image_path = ""
image_prompt = ""
generation_mode = False
generation_model_name = "gpt-image-1"
depth_estimation_model_path = None  # facebook/VGGT-1B
pose_estimation_model_path = None  # usyd-community/vitpose-plus-base


In [5]:
model = VisionReasonerModel(
    reasoning_model_path=model_path, 
    task_router_model_path=task_router_model_path, 
    segmentation_model_path=segmentation_model_path,
    depth_estimation_model_path=depth_estimation_model_path,
    pose_estimation_model_path=pose_estimation_model_path
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.08it/s]
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
